<a href="https://colab.research.google.com/github/seongkyu-lim/Graduation-thesis/blob/master/YOLOv5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
from IPython.display import Image, clear_output  # to display images

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.9.0+cu102 (Tesla T4)


In [5]:
# mount gdrive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Inference
이미 trained된 모델인 yolov5x,s,m,l 중 성능이 가장 좋은 xlarge모델로 inference 해보았습니다.

In [ ]:
import torch

%ls

#절대경로로 읿력해주어야 파일 위치 인식한다.
!python /content/yolov5/detect.py --weight yolov5x.pt --img 1280 --conf 0.4 --save-conf --save-txt --save-crop --source /content/yolov5/data/images/train

CONTRIBUTING.md  Dockerfile  LICENSE    requirements.txt*  utils/
data/            export.py   models/    train.py           val.py
detect.py        hubconf.py  README.md  tutorial.ipynb
100% 755k/755k [00:00<00:00, 31.8MB/s]
detect: weights=['yolov5x.pt'], source=/content/yolov5/data/images/train, imgsz=[1280, 1280], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=True, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-438-g27a4736 torch 1.9.0+cu102 CUDA:0 (Tesla T4, 15109.75MB)

100% 168M/168M [00:01<00:00, 93.1MB/s]

Fusing layers... 
Model Summary: 476 layers, 87730285 parameters, 0 gradients
image 1/45 /content/yolov5/data/images/train/frame0.jpg: 736x1280 9 persons, Done. (0.125s)
image 2/45 /content/yolov5/data/images/train/frame1.jpg: 736x1280 14 p

In [ ]:
# 이미지로 확인해보기. 
# 영상으로 저장되서 불가능.
Image(filename='runs/detect/exp/', width=600)

ValueError: ignored

In [ ]:
# inference된 영상 다운로드 받기.
from google.colab import files
files.download('./inference/output/ARS-CHEL-0.mp4')

# 또는 구글드라이브로 옮겨서 확인하기.

#Train

  train하기

##Challenge

- 양 팀을 다르게 인식해야한다.
- 심판과 공또한 인식해야한다.

##1.opencv를 활용한 영상 처리

In [1]:
%pip install opencv-python
%pip install numpy
%pip install matplotlib

In [10]:
# -*- coding: utf-8 -*-
__author__ = 'Seran'
 
import cv2
 
# 영상의 의미지를 연속적으로 캡쳐할 수 있게 하는 class
vidcap = cv2.VideoCapture('/content/drive/MyDrive/졸업논문/20210822-ARS-CHE-EPL_1-1080.mkv')
 
count = 0

while(vidcap.isOpened()):
    ret, image = vidcap.read()
    
    if(int(vidcap.get(1)) % 200 == 0):
        print('Saved frame number : ' + str(int(vidcap.get(1))))
        cv2.imwrite("/content/yolov5/data/images/train3/frame%d.jpg" % count, image) # data/images/train에 저장.
        print('Saved frame%d.jpg' % count)
        count += 1
 
vidcap.release()

KeyboardInterrupt: ignored

In [11]:
%mv /content/yolov5/data/images/train3 /content/drive/MyDrive/졸업논문


## 2.yolov5로 train 진행

In [ ]:
# Download COCO128
torch.hub.download_url_to_file('https://github.com/ultralytics/yolov5/releases/download/v1.0/coco128.zip', 'tmp.zip')
!unzip -q tmp.zip -d ../datasets && rm tmp.zip

  0%|          | 0.00/6.66M [00:00<?, ?B/s]

In [3]:

'''
# Tensorboard  (optional)
%load_ext tensorboard
%tensorboard --logdir runs/train
'''
# Weights & Biases(wandb)  (optional)
%pip install -q wandb
import wandb
wandb.login()


     |████████████████████████████████| 1.7 MB 5.2 MB/s 
     |████████████████████████████████| 133 kB 44.3 MB/s 
     |████████████████████████████████| 180 kB 46.0 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [5]:
#구글 드라이브에서 yolo모델 데이터셋 폴더로 커스텀 데이터 이동.
%cp /content/drive/MyDrive/졸업논문/train /content/yolov5/data/images
%cp /content/drive/MyDrive/졸업논문/train_labels /content/yolov5/data/labels
%cp /content/drive/MyDrive/졸업논문/dataset.yaml /content/yolov5/data

In [28]:
import torch
torch.cuda.empty_cache()

torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |\n|       from large pool |       0 B  |       0 B  |

In [29]:
# dataset.yaml에 훈련, valid를 위한 데이터셋 저장 경로 정보 있음.
# dataset.yaml에 경로는 절대경로로

!python train.py --img 640 --batch 10 --epochs 5 --data dataset.yaml --weights yolov5x.pt --cache

wandb: Currently logged in as: seongkyu (use `wandb login --relogin` to force relogin)
train: weights=yolov5x.pt, cfg=, data=dataset.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=5, batch_size=10, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=runs/train, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=-1, artifact_alias=latest, local_rank=-1, freeze=0, patience=100
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v5.0-454-g9febea7 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_

In [ ]:
%rm -rf /content/yolov5/data/labels/train 
%rm -rf /content/yiolov5/data/labels/valid

## 3.train을 통해 얻은 best.pt을 이용하여 다시 inference 해보기 

In [13]:
%cp /content/drive/MyDrive/졸업논문/best.pt /content/yolov5

In [14]:
%cp /content/drive/MyDrive/졸업논문/ARS-CHEL-0.2.mp4 /content/yolov5/data/images/train2

In [16]:
%cp /content/yolov5/data/labels/train/classes.txt /content/yolov5/data/images/train2

In [34]:
!python /content/yolov5/detect.py --weight best.pt --img 640 --conf 0.4  --save-txt --save-crop --source /content/yolov5/data/images/train

detect: weights=['best.pt'], source=/content/yolov5/data/images/train, imgsz=[640, 640], conf_thres=0.4, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=True, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False
YOLOv5 🚀 v5.0-454-g9febea7 torch 1.9.0+cu102 CUDA:0 (Tesla K80, 11441.1875MB)

Fusing layers... 
Model Summary: 476 layers, 87340003 parameters, 0 gradients, 217.5 GFLOPs
image 1/33 /content/yolov5/data/images/train/frame102.jpg: 384x640 Done. (0.158s)
image 2/33 /content/yolov5/data/images/train/frame39.jpg: 384x640 Done. (0.151s)
image 3/33 /content/yolov5/data/images/train/frame41.jpg: 384x640 Done. (0.138s)
image 4/33 /content/yolov5/data/images/train/frame44.jpg: 384x640 Done. (0.138s)
image 5/33 /content/yolov5/data/images/train/frame45.jpg: 384x640 Done. (0.138s)
image 6/3